This notebook shows how to use the simple model serving functionality found in `tfe.serving`.

# Setup

In [1]:
from collections import OrderedDict

import tensorflow as tf
import tf_encrypted as tfe

Turn on TFE profling flags since we want to inspect with TensorBoard.

In [2]:
TENSORBOARD_DIR = "/tmp/tensorboard"

tfe.set_tfe_trace_flag(True)
tfe.set_tfe_events_flag(True)
tfe.set_log_directory(TENSORBOARD_DIR)

!rm -rf {TENSORBOARD_DIR}

INFO:tf_encrypted:Writing trace files
INFO:tf_encrypted:Writing event files for each run with a tag
INFO:tf_encrypted:Writing event and trace files to '/tmp/tensorboard'


# Protocol

We first configure the protocol we will be using, as well as the servers on which we want to run it.

Note that the configuration is saved to file as we will be needing it in the client as well.

In [3]:
players = OrderedDict([
    ('server0', 'localhost:4000'),
    ('server1', 'localhost:4001'),
    ('server2', 'localhost:4002'),
])

config = tfe.RemoteConfig(players)
config.save('/tmp/tfe.config')

In [4]:
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.Pond())

## Launching servers

Before actually serving the computation below we need to launch TensorFlow servers in new processes. Run the following in three different terminals. You may have to allow Python to accept incoming connections.

In [5]:
for player_name in players.keys():
    print("python -m tf_encrypted.player --config /tmp/tfe.config {}".format(player_name))

python -m tf_encrypted.player --config /tmp/tfe.config server0
python -m tf_encrypted.player --config /tmp/tfe.config server1
python -m tf_encrypted.player --config /tmp/tfe.config server2


# Computation

We then define the computation we want to run. These will happen on private tensors on the servers defined above.

Note that the only single-tensor computations are currently supported.

In [6]:
input_shape = (5, 5)
output_shape = (5, 5)

def computation(x):
    return x * x

We can then set up a new `tfe.serving.QueueServer` to serve this computation.

In [7]:
server = tfe.serving.QueueServer(
    input_shape=input_shape,
    output_shape=output_shape,
    computation_fn=computation)

# Serving

With all of the above in place we can finally connect to our servers, push our graph to them, and start serving computations.

In [8]:
sess = tfe.Session(disable_optimizations=True)

tf.Session.reset(sess.target)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
  optimizer_options {
    opt_level: L0
  }
  rewrite_options {
    arithmetic_optimization: OFF
  }
}



In [ ]:
def step_fn():
    print("Next")

server.run(
    sess,
    num_steps=5,
    step_fn=step_fn)

Next


At this point we switch to the client notebook to run computations.